In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from tqdm import tqdm

In [ ]:
# dataset and loader
# MNIST
train_dataset = datasets.MNIST(root="./mnist_data/", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="./mnist_data/", train=False, transform=transforms.ToTensor(), download=False)

# dataloader
batch_size = 16
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class VAE(nn.Module):

    def __init__(self, x_dim, z_dim):
        super().__init__()

        self.x_dim = x_dim
        self.z_dim = z_dim

        # encoder
        self.fc_enc = nn.Linear(x_dim, 512)
        self.fc_mu = nn.Linear(512, z_dim)
        self.fc_log_var = nn.Linear(512, z_dim)

        # decoder
        self.fc_dec = nn.Sequential(
            nn.Linear(z_dim, 512),
            nn.ReLU(),
            nn.Linear(512, x_dim),
            nn.Sigmoid(),
        )

    def encode(self, x):
        x = torch.flatten(x, start_dim=2)
        x = self.fc_enc(x)
        mu = self.fc_mu(x)
        log_var = self.fc_log_var(x)
        return mu, log_var
    
    def decode(self, z):
        return self.fc_dec(z)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        recon_x = self.decode(z)
        return recon_x, x, mu, log_var
    
    def loss_function(self, recon_x, x, mu, log_var):
        kld_weight = 1
        recon_loss = F.mse_loss(recon_x, x.view(-1, 784))
        kld_loss = -0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp())
        loss = recon_loss + kld_weight * kld_loss
        return {'loss': loss, 'reconstruction_loss': recon_loss, 'KLD': -kld_loss}
    
    def sample(self, num_samples, device):
        z = torch.randn(num_samples, self.z_dim).to(device)
        samples = self.decode(z)
        return samples
    
    def generate(self, x):
        return self.forward(x)[0]


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VAE(x_dim=784, z_dim=2).to(device)
print(model)

In [ ]:
# settings of training
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epoch = 50

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for i, (x, _) in tqdm(enumerate(train_loader)):
        x = x.to(device)
        recon_x, x, mu, log_var = model(x)
        
        loss = model.loss_function(recon_x, x, mu, log_var)["loss"]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    print(f"Epoch: {epoch:3d}, loss: {train_loss / len(train_loader.dataset):.3f}")

In [ ]:
def test():
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for x, _ in test_loader:
            x = x.to(device)
            recon_x, x, mu, log_var = model(x)
            test_loss += model.loss_function(recon_x, x, mu, log_var)["loss"]
    
    print(f"---> Test loss: {test_loss / len(test_loader.dataset):.3f}")

In [ ]:
for epoch in range(1, num_epoch+1):
    train(epoch)
    test()